In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/kaggle/input/filtered-and-translated-nlp/filr.csv')

In [ ]:
X = data['Translated']
y = data['Label']

In [ ]:
X

In [ ]:
y

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def generate_embeddings(texts):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
            outputs = model(**inputs)
            pooled_output = outputs.pooler_output.squeeze().numpy()
            embeddings.append(pooled_output)
    return np.array(embeddings)

# Generate embeddings
print("Generating BERT embeddings...")
X_embeddings = generate_embeddings(X)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)

# Train an SVM classifier
print("Training SVM...")
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Follow a repo 
For finetuning

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

In [ ]:
data = pd.read_csv('/kaggle/input/filtered-and-translated-nlp/filr.csv')

In [ ]:
X = data['Translated']
y = data['Label']

In [ ]:
y.value_counts(normalize = True)

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(X, y, 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=y)

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [ ]:
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
# sample data
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]

# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)

In [ ]:
print(sent_id)

# Tokenization

In [ ]:
seq_len = [len(i.split()) for i in train_text]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
max_seq_len = max(seq_len)

In [ ]:
max_seq_len

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:

# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:


# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,4)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):
    # Pass the inputs to the model
        outputs = self.bert(sent_id, attention_mask=mask)
        cls_hs = outputs.pooler_output
        
        # Pass through fully connected layers
        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
    
        return x

In [ ]:
print(device)

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [ ]:

# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

In [ ]:

from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)

print(class_wts)

In [ ]:

# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [ ]:

# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:


# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()
     

# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

# ChatGPT

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Custom Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts.iloc[index]  # Using .iloc for safe indexing
        label = self.labels.iloc[index]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }


# Define Model
class BERTClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.pooler_output)
        return logits

# Prepare Data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TextDataset(X_train, y_train, tokenizer)
test_dataset = TextDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Initialize Model and Parameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERTClassifier(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Training Loop
def train_model(epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# Evaluation
def evaluate_model():
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            predictions = torch.argmax(outputs, dim=1)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Run Training and Evaluation
train_model(25)
evaluate_model()


In [ ]:
import os

def save_model(model, path='/kaggle/working/model_state_dict.pt'):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(model.state_dict(), path)
    print(f"Model state dict saved to {path}")


In [ ]:
save_model(model)

In [ ]:
def save_tokenizer(tokenizer, path='/kaggle/working/tokenizer'):
    os.makedirs(path, exist_ok=True)
    tokenizer.save_pretrained(path)
    print(f"Tokenizer saved to {path}")


In [ ]:
save_tokenizer(tokenizer)

In [ ]:
import numpy as np

def generate_embeddings(texts, tokenizer, model, batch_size=16, max_len=512):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    embeddings = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]

            inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt')
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            
            outputs = model.bert(input_ids=input_ids, attention_mask=attention_mask)

            # If pooler_output is available, use it; otherwise, use mean of last hidden state
            if hasattr(outputs, 'pooler_output'):
                batch_embeddings = outputs.pooler_output.cpu().numpy()
            else:
                batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

            embeddings.extend(batch_embeddings)
    
    return np.array(embeddings)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, y_test are available
X_train_embeddings = generate_embeddings(X_train.tolist(), tokenizer, model)
X_test_embeddings = generate_embeddings(X_test.tolist(), tokenizer, model)

# Train the SVM
svm_classifier = SVC(kernel='linear', class_weight = 'balanced')
svm_classifier.fit(X_train_embeddings, y_train)

# Predict and Evaluate
y_pred = svm_classifier.predict(X_test_embeddings)
accuracy = accuracy_score(y_test, y_pred)

print(f"Test Accuracy with SVM: {accuracy * 100:.2f}%")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Predict using SVM
y_pred = svm_classifier.predict(X_test_embeddings)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

# AUC-ROC Score
if len(set(y_test)) <= 2:
    auc_score = roc_auc_score(y_test, svm_classifier.decision_function(X_test_embeddings))
    print(f"AUC-ROC Score: {auc_score:.2f}")

    # Plot ROC Curve
    fpr, tpr, _ = roc_curve(y_test, svm_classifier.decision_function(X_test_embeddings))
    plt.figure(figsize=(8,6))
    plt.plot(fpr, tpr, color='blue', label=f"AUC = {auc_score:.2f}")
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.show()
else:
    print("AUC-ROC is not applicable for multi-class classification.")


In [ ]:
print('hi')

In [ ]:
misclassified_indices = np.where(y_test != y_pred)[0]
for idx in misclassified_indices[:10]:  # Inspect first 10 misclassified samples
    print(f"Text: {X_test.iloc[idx]}")
    print(f"True Label: {y_test.iloc[idx]}, Predicted Label: {y_pred[idx]}\n")

# Early Stopping

In [ ]:
class BERTClassifierWithAttention(nn.Module):
    def __init__(self, num_classes=4):
        super(BERTClassifierWithAttention, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.pooler_output)
        
        # Extract attention weights (already normalized)
        attentions = outputs.attentions  # Tuple of attention weights for each layer
        
        # Compute attention scores (unnormalized)
        attention_scores = []
        for layer in self.bert.encoder.layer:
            attn = layer.attention.self
            q = attn.query(input_ids)
            k = attn.key(input_ids)
            scores = torch.matmul(q, k.transpose(-2, -1)) / (q.size(-1) ** 0.5)
            attention_scores.append(scores.detach())  # Store unnormalized scores
        
        return logits, attentions, attention_scores

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_attention_scores(scores, title="Attention Scores"):
    # Normalize scores for visualization
    scores = scores[0].detach().numpy()  # Shape: (num_heads, seq_len, seq_len)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(scores[0], annot=True, fmt=".2f", cmap="viridis")  # First head
    plt.title(title)
    plt.xlabel("Keys")
    plt.ylabel("Queries")
    plt.show()

# Attention

## Initialize bert and set output attentions to True

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class BERTClassifierWithAttention(nn.Module):
    def __init__(self, num_classes=4):
        super(BERTClassifierWithAttention, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.pooler_output)
        
        # Extract attention weights (normalized scores)
        attentions = outputs.attentions  # Tuple of attention weights for each layer
        
        # Extract Q, K, V matrices manually
        qkv_values = []
        for layer in self.bert.encoder.layer:
            attn = layer.attention.self
            q = attn.query(outputs.last_hidden_state)  # Query projection
            k = attn.key(outputs.last_hidden_state)    # Key projection
            v = attn.value(outputs.last_hidden_state)  # Value projection
            qkv_values.append((q.detach(), k.detach(), v.detach()))
        
        return logits, attentions, qkv_values

## Load the dataset and set up the finetuning environment

In [ ]:
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import AdamW

# Custom Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts.tolist() if hasattr(texts, 'tolist') else texts
        self.labels = labels.tolist() if hasattr(labels, 'tolist') else labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Prepare Data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TextDataset(X_train, y_train, tokenizer)
val_dataset = TextDataset(X_val, y_val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERTClassifierWithAttention(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

## Train it
> Run a few epochs on our dataset

In [ ]:
# Training Loop
def train_model(epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            logits, _, _ = model(input_ids, attention_mask)  # Ignore attentions and QKV during training
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# Run Training
train_model(epochs=20)



## Generate Embeddings from finetuned model

In [ ]:
def generate_embeddings(texts, tokenizer, model, batch_size=16, max_len=512):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    embeddings = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt')
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            
            outputs = model.bert(input_ids=input_ids, attention_mask=attention_mask)
            batch_embeddings = outputs.pooler_output.cpu().numpy()  # Use pooler_output
            embeddings.extend(batch_embeddings)
    
    return embeddings

# Generate embeddings for train and test sets
X_train_embeddings = generate_embeddings(X_train.tolist(), tokenizer, model)
X_val_embeddings = generate_embeddings(X_val.tolist(), tokenizer, model)

## Save model

## Classify embeddings using SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Train the SVM
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_embeddings, y_train)

# Predict and Evaluate
y_pred = svm_classifier.predict(X_val_embeddings)
accuracy = accuracy_score(y_val, y_pred)

print(f"Validation Accuracy with SVM: {accuracy * 100:.2f}%")

## Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve


# Confusion Matrix
conf_matrix = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:\n")
print(classification_report(y_val, y_pred))

## Analyze misclassified

In [ ]:
misclassified_indices = np.where(y_val != y_pred)[0]
for idx in misclassified_indices[:10]:  # Inspect first 10 misclassified samples
    print(f"Text: {X_val.iloc[idx]}")
    print(f"True Label: {y_val.iloc[idx]}, Predicted Label: {y_pred[idx]}\n")

## Extract attentions and Q, K and V values

In [ ]:
def extract_attention_and_qkv(model, tokenizer, text):
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        _, attentions, qkv_values = model(input_ids, attention_mask)
    
    return attentions, qkv_values

# Example usage
text = """
#RussiansAreOurBrothers #UkrainiansAreOurBrothers.
"""
attentions, qkv_values = extract_attention_and_qkv(model, tokenizer, text)

# Inspect Q, K, V values for the first layer
q, k, v = qkv_values[0]
print("Query Matrix:", q)
print("Key Matrix:", k)
print("Value Matrix:", v)

In [ ]:
print(attentions)

## Visualizing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch

def generate_word_embeddings(text, tokenizer, model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state.cpu().numpy()[0]

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    return tokens, last_hidden_state

def plot_word_embeddings(tokens, embeddings):
    # Reduce dimensions to 2D using PCA
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)

    # Plot the embeddings
    plt.figure(figsize=(12, 8))
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], color='blue')

    for i, token in enumerate(tokens):
        plt.annotate(token, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]), fontsize=10)

    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.title('2D Word Embedding Visualization')
    plt.grid()
    plt.show()

# Example usage
text = "#RussiansAreOurBrothers #UkrainiansAreOurBrothers."
tokens, embeddings = generate_word_embeddings(text, tokenizer, model)
plot_word_embeddings(tokens, embeddings)


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Load non-finetuned BERT model and tokenizer
non_finetuned_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
non_finetuned_model = BertModel.from_pretrained('bert-base-uncased')

def generate_non_finetuned_embeddings(text, tokenizer, model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, add_special_tokens=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state.squeeze(0).cpu().numpy()

    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0).tolist())
    return embeddings, tokens

def plot_embeddings_with_labels(embeddings, tokens, title):
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)

    plt.figure(figsize=(10, 8))
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], marker='o', c='blue')

    for i, token in enumerate(tokens):
        plt.annotate(token, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]), fontsize=9)

    plt.title(title)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid(True)
    plt.show()

# Example text
text = "#RussiansAreOurBrothers #UkrainiansAreOurBrothers."

# Generate and plot embeddings for non-finetuned BERT
non_finetuned_embeddings, tokens = generate_non_finetuned_embeddings(text, non_finetuned_tokenizer, non_finetuned_model)
plot_embeddings_with_labels(non_finetuned_embeddings, tokens, 'Non-Finetuned BERT Embeddings with Annotations')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_attention_scores(attentions, layer_idx=0, head_idx=0):
    """
    Plots attention scores for a specific layer and head.
    """
    # Extract attention weights for the specified layer and head
    attention_weights = attentions[layer_idx][0, head_idx].detach().cpu().numpy()  # Shape: (seq_len, seq_len)
    
    # Plot heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(attention_weights, annot=True, fmt=".2f", cmap="viridis")
    plt.title(f"Attention Scores (Layer {layer_idx}, Head {head_idx})")
    plt.xlabel("Keys")
    plt.ylabel("Queries")
    plt.show()

### Markdown

In [ ]:

# Plot attention scores for the first layer and first head
plot_attention_scores(attentions, layer_idx=0, head_idx=0)

### Query Values

In [ ]:
def plot_query_values(q, tokenized_text, layer_idx=0, head_idx=0):
    """
    Plots query values for a specific layer and head.
    """
    # Extract query values for the specified layer and head
    q_values = q[layer_idx][0, :, head_idx].detach().cpu().numpy()  # Shape: (seq_len,)
    
    # Decode tokenized text
    tokens = tokenizer.convert_ids_to_tokens(tokenized_text['input_ids'][0])
    
    # Plot bar chart
    plt.figure(figsize=(12, 6))
    plt.bar(tokens, q_values, color='skyblue')
    plt.title(f"Query Values (Layer {layer_idx}, Head {head_idx})")
    plt.xlabel("Tokens")
    plt.ylabel("Query Value")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_qkv_matrix_with_tokens(matrix, tokens, title="Matrix", cmap="viridis"):
    """
    Plots a heatmap of the given Q, K, or V matrix with token labels.
    """
    # Convert matrix to numpy array if it's a PyTorch tensor
    if isinstance(matrix, torch.Tensor):
        matrix = matrix.detach().cpu().numpy()
    
    # Plot heatmap
    plt.figure(figsize=(12, 6))
    sns.heatmap(matrix, cmap=cmap, yticklabels=tokens)
    plt.title(title)
    plt.xlabel("Hidden Dimensions")
    plt.ylabel("Tokens")
    plt.tight_layout()
    plt.show()

In [ ]:



# Tokenize the input text
text = """
#RussiansAreOurBrothers #UkrainiansAreOurBrothers.
"""
encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)

# Extract tokens
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])  # List of tokens
print("Tokens:", tokens)

In [ ]:
# Extract attention scores and Q, K, V values
attentions, qkv_values = extract_attention_and_qkv(model, tokenizer, text)

# Inspect Q, K, V values for the first layer
q, k, v = qkv_values[0]
print("Query Matrix Shape:", q.shape)
print("Key Matrix Shape:", k.shape)
print("Value Matrix Shape:", v.shape)

In [ ]:
# Move tensors to CPU and convert to numpy arrays
q_matrix = q[0, 0].detach().cpu().numpy()  # Shape: (seq_len, head_dim)
k_matrix = k[0, 0].detach().cpu().numpy()  # Shape: (seq_len, head_dim)
v_matrix = v[0, 0].detach().cpu().numpy()  # Shape: (seq_len, head_dim)

print("Query Matrix Shape:", q_matrix.shape)
print("Key Matrix Shape:", k_matrix.shape)
print("Value Matrix Shape:", v_matrix.shape)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_attention_scores(attentions, tokens, layer_idx=0, head_idx=0):
    """
    Plots attention scores for a specific layer and head with tokens as labels.
    attentions: Attention weights from BERT output (shape: [layers, batch, heads, seq_len, seq_len])
    tokens: Tokenized input (list of token strings)
    layer_idx: Index of the layer to visualize
    head_idx: Index of the head to visualize
    """
    # Extract attention weights for the specified layer and head
    attention_weights = attentions[layer_idx][0, head_idx].detach().cpu().numpy()

    # Plot heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        attention_weights, 
        annot=True, 
        fmt=".2f", 
        cmap="viridis",
        xticklabels=tokens, 
        yticklabels=tokens
    )
    plt.title(f"Attention Scores (Layer {layer_idx}, Head {head_idx})")
    plt.xlabel("Keys (Words being attended to)")
    plt.ylabel("Queries (Words attending)")
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.show()

# Example usage
# Assuming `attentions` is the attention tensor from BERT and `tokens` is the tokenized input
plot_attention_scores(attentions, tokens, layer_idx=0, head_idx=0)
